In [2]:
IMG_SIZE = 528
BATCH_SIZE = 16
EPOCHS = 50
VAL_SPLIT = 0.2

# Data Loading

In [3]:
import os

dataset_path = "../../data-collection/image-backend/saved_images"
hazard_path = os.path.join(dataset_path, "hazard/")
non_hazard_path = os.path.join(dataset_path, "non-hazard/")

In [4]:
import tensorflow as tf

hazard_images = tf.keras.utils.image_dataset_from_directory(
    hazard_path,
    labels=None,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=None,
)

hazard_images.class_name = [1]

Found 3 files.


In [5]:
non_hazard_images = tf.keras.utils.image_dataset_from_directory(
    non_hazard_path,
    labels=None,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=None
)

non_hazard_images.class_name = [0]

Found 4 files.


In [6]:
train_ds = hazard_images.concatenate(non_hazard_images)

In [62]:
print(type(train_ds))

<class 'tensorflow.python.data.ops.concatenate_op._ConcatenateDataset'>


In [7]:
from tensorflow.keras.applications.efficientnet import preprocess_input
train_ds.shuffle(buffer_size=1024).batch(BATCH_SIZE)

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))

val_ds = train_ds.take(int(VAL_SPLIT * len(train_ds)))
train_ds = train_ds.skip(int(VAL_SPLIT * len(train_ds)))

TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.<lambda>() missing 1 required positional argument: 'y'


# Model Fine-tuning

In [8]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB6
import tensorflow as tf


def build_model():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    model = EfficientNetB6(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(1, activation="sigmoid", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model

In [9]:
model = build_model()
model.summary()

Model: "EfficientNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 528, 528,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 528, 528,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 528, 528,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 528, 528,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 529, 529,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 264, 264,  │      1,512 │ stem_conv_pad[0]… │
│                     │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 264, 264,  │        224 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 264, 264,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 264, 264,  │        504 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 264, 264,  │        224 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 264, 264,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 56)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 56)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 56)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 14)  │        798 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 56)  │        840 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 264, 264,  │          0 │ block1a_activati… │
│ (Multiply)          │ 56)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 264, 264,  │      1,792 │ block1a_se_excit

 Total params: 40,971,664 (156.29 MB)

 Trainable params: 6,913 (27.00 KB)

 Non-trainable params: 40,964,751 (156.27 MB)

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, verbose=2)

In [11]:
model.save("../models/EfficientNET.keras")